<a href="https://colab.research.google.com/github/machine-perception-robotics-group/ImageProcessingGoogleColabNotebooks/blob/master/05_binary_image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. 二値化，二値画像処理

講義で説明する画像処理の方法について，google colaboratoryを利用して演習する．
google colaboratoryは，クラウドで実行する Jupyter ノートブック環境である.
google coraboratoryについては，[ここ](https://www.tdi.co.jp/miso/google-colaboratory-gpu)や[ここ](https://www.codexa.net/how-to-use-google-colaboratory/)を参考にすること．

下記のプログラムを実行すると，画像の二値化や二値化した画像の処理を行う．

## 準備
プログラムの動作に必要なデータをダウンロードし，zipファイルを解凍する．

In [ ]:
!wget -q https://raw.githubusercontent.com/machine-perception-robotics-group/ImageProcessingGoogleColabNotebooks/master/image1.zip
!unzip -q image1.zip
!ls
!ls ./image1/

## 画像の読み込みと表示
必要なパッケージをインポートし，画像を表示する．

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# カラー画像の読み込み
img_src = cv2.imread('./image1/woman-color.jpg')
img_rgb = cv2.cvtColor(img_src, cv2.COLOR_BGR2RGB)
# グレースケール画像の読み込み
img_gray = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)

# 画像を表示
plt.imshow(img_rgb)
plt.title("color image")
plt.show()

plt.imshow(img_gray, cmap="gray", clim=(0, 255))
plt.title("grayscale image")
plt.show()

## グレースケール変換

ここでは，複数の方法でカラー画像をグレースケール画像へと変換する．

1. 彩度を0にする方法では，各チャンネルの値の最大値と最小値の平均を求めることでグレースケール変換を行う．
2. RGBの平均値を用いることでグレースケール変換を行う．
3. YUVを用いる方法では，$0.299 \times R + 0.587 \times G + 0.114 \times B$のように値を計算することで，グレースケール変換を行う．

変換結果より，用いる変換方法によりグレースケール画像の明るさ等が異なることがわかる．

In [ ]:
img = img_rgb.copy()

# 高さ・幅・チャンネル数の取得
height, width, channel = img.shape

# 要素が全て0の3次元配列を作成
image_array1 = np.zeros((height, width), dtype=np.uint8)
image_array2 = np.zeros((height, width), dtype=np.uint8)
image_array3 = np.zeros((height, width), dtype=np.uint8)

# RGBをソート
img_sort = np.sort(img, axis=2)

# 配列の各要素に値（画素値）を代入
for h in range(0, height):
    for w in range(0, width):
      # 1. 彩度0
      image_array1[h, w] = (img_sort[h, w, 0] * 1 + img_sort[h, w, 2] * 1) / 2.0
      # 2. RGBの平均値
      image_array2[h, w] = (img[h, w, 0] * 1 + img[h, w, 1] * 1 + img[h, w, 2] * 1) / 3
      # 3. YUV(0.299*R + 0.587*G + 0.114*B)色ごとの明るさの感じ方の違いを反映
      image_array3[h, w] = (img[h, w, 0] * 0.299) + (img[h, w, 1] * 0.587) + (img[h, w, 2] * 0.114) 

# 画像の表示
plt.imshow(image_array1, cmap="gray", clim=(0, 255))
plt.title("saturation=0")
plt.show()

plt.imshow(image_array2, cmap="gray", clim=(0, 255))
plt.title("average")
plt.show()

plt.imshow(image_array3, cmap="gray", clim=(0, 255))
plt.title("YUV")
plt.show()

## 二値化

次に，画像の二値化を行う．
画像の二値化には主に次のような方法がある．
* 手動で閾値を選択する方法
* pタイル法
* モード法
* 判別分析法

以下では，それぞれの方法を用いて画像の二値化を行う．



### 手動で閾値を選択する方法

この方法では，閾値を手動で設定し．二値化を行う．
画素値が閾値未満の場合には0に閾値以上の場合には255に変更することで二値化を行う．

In [ ]:
#上で読み込んだ低コントラストグレースケール画像をコピー
img = img_gray.copy()

# 高さ・幅の取得
height, width = img.shape

# 閾値の設定
thresh = 128

for i in range(height):
    for j in range(width):
        if img[i][j] < thresh:  # 画素値が閾値未満の場合
            img[i][j] = 0
        else:                   # それ以外
            img[i][j] = 255

# 2値化した画像の表示
img_bin_manual = img
plt.imshow(img_bin_manual, cmap="gray", clim=(0, 255))

### pタイル法


pタイル法では，画素値の低いところから頻度値を計算し，予測された画素数を超えた時の画素値をしきい値として決定する．

まず，画像の濃淡ヒストグラムを計算し，二値化する領域の割合 (`ratio`) を決定する．
その後，画像サイズと比率から，予測される画素数`th`を計算する．

その後，ヒストグラムにカウントされた画素数を一つづつfor文を用いて加算していき，
加算した画素数が予測される画素数`th`を超えた時の画素値を閾値`threshold`として保存する．

最後に得られた`threshold`を用いて画像を二値化する．



In [ ]:
#上で読み込んだ低コントラストグレースケール画像をコピー
img = img_gray.copy()

# 高さ・幅の取得
height, width = img.shape

# 画像の画素数を計算
imgsize = height * width

# ヒストグラムを計算するための，配列を用意
histogram = np.arange(256)

# for文で1つずつ画素値にアクセスする
for h in range(height):
  for w in range(width):
    # 1画素ずつ画素値を見ながら該当する要素（bin）に投票
    histogram[img[h][w]] += 1

# 領域の割合（比率）を決定
ratio = 0.4

th = height * width * ratio
th_sum = 0
for ti in range(256):
  th_sum += histogram[ti]
  if th_sum > th:
    threshold = ti
    break

print("THRESHOLD =", threshold)

# 画像の二値化と表示
img_bin_ptile = img.copy()
for i in range(height):
  for j in range(width):
    # 閾値未満なら
    if img_bin_ptile[i][j] < threshold:
      img_bin_ptile[i][j] = 0
    # それ以外
    else:
      img_bin_ptile[i][j] = 255

plt.imshow(img_bin_ptile, cmap="gray", clim=(0, 255))
plt.show()

### モード法

モード法は濃淡ヒストグラムの谷となる画素値を閾値として決定する方法である．

まず，濃淡ヒストグラムの作成を行う．
その後，ヒストグラムの平滑化を行う．
ヒストグラムの平滑化は02「空間フィルタリング」で用いた移動平均フィルタと同様の処理をヒストグラム上で行う．

次に，平坦化したヒストグラムを用いて，山（極大値）を探索する．
探索では，for文でヒストグラムの値を順番に確認していく．
その際，`i`番目のbinの値 < `i+1`番目のbinの値 > `i+2`番目のbinの値となる場合に山と判断し，山に該当するbinの値を記録しておく．

その後，もっとも値の大きな2つの山を選択し，その山の間の値がもっとも小さいbinを谷（極小値）として選択し，この画素値を閾値とする．

元画像の濃淡ヒストグラムと平滑化したヒストグラムを表示する．
平滑化したヒストグラムには，探索によって発見された山と谷（閾値）をプロットする．

この結果より，平滑化によりヒストグラムの細かな起伏がなくなり，大きな山や谷を探索できていることがわかる．

In [ ]:
#上で読み込んだ低コントラストグレースケール画像をコピー
img = img_gray.copy()

# 高さ・幅の取得
height, width = img.shape

# 画像の画素数を計算
imgsize = height * width

# ヒストグラムの計算
histogram = np.arange(256)
for h in range(height):
  for w in range(width):
    # 1画素ずつ画素値を見ながら該当する要素（bin）に投票
    histogram[img[h][w]] += 1

# ヒストグラムの平滑化
hist_sm = histogram.copy()
for i in range(256 - 4):
  hist_sm[i] = np.mean(histogram[i:i+5])

# 平滑化したヒストグラムを用いて山（極大値）を探索
maxima_list = np.zeros(256)
for i in range(256 - 3):
  if hist_sm[i] < hist_sm[i+1] and hist_sm[i+2] < hist_sm[i+1]:
    maxima_list[i+1] = hist_sm[i+1]

# 値の高い極大値２つを選択する
maxima_indices = np.argsort(maxima_list)
peak = np.sort(maxima_indices[-2:])
print("peak:", peak)

# 極大値の間でもっとも頻度が低い画素値（閾値）を求める
minima = [-1, 20000]
for i in range(peak[0], peak[1]):
  if hist_sm[i] < minima[1]:
    minima = [i, hist_sm[i]]

threshold = minima[0]
print("minima (threshold):", threshold)

# 平滑化したヒストグラムの表示
plt.figure(figsize=(14, 4))
plt.subplot(121)
plt.bar(range(256), histogram, width=1.0)
plt.title("original histogram")
plt.subplot(122)
plt.bar(range(256), hist_sm, width=1.0)
plt.title("smoothed histogram")
plt.axvline(peak[0], color="orange", label='peak1')
plt.axvline(peak[1], color="red", label='peak2')
plt.axvline(threshold, color="green", label='threshold')
plt.legend()
plt.show()

上のモード法で求めた閾値を用いて，画像を二値化する．

In [ ]:
img_bin_mode = img.copy()

for i in range(height):
  for j in range(width):
    # 閾値未満なら
    if img_bin_mode[i][j] < threshold:
      img_bin_mode[i][j] = 0
    # それ以外
    else:
      img_bin_mode[i][j] = 255

plt.imshow(img_bin_mode, cmap="gray", clim=(0, 255))
plt.show()

### 判別分析法

判別分析法では，閾値で全景と背景を分割し，2つのクラス間の分離度が最大となる画素値を閾値とする方法である．

まず，濃淡ヒストグラムを作成する．

次に，探索中にもっとも良かった閾値と分離度を保存するためのリスト`s_max`を用意する．

その後，for文を用いて，閾値を変化させ，クラスの分離度を計算していき，もっとも分離度が高い閾値を探索する．


In [ ]:
#上で読み込んだ低コントラストグレースケール画像をコピーし高さ・幅を取得
img = img_gray.copy()
height, width = img.shape

# 画像の画素数を計算
imgsize = height * width

# ヒストグラムの計算
histgram = np.arange(256)
for h in range(height):
  for w in range(width):
    histgram[img[h][w]] += 1

# 閾値と分離度を保存するためのリストを用意
s_max = [0, -10]

for th in range(1, 256 - 1):
  # クラス1とクラス2の画素数を計算
  n1 = sum(histgram[:th])
  n2 = sum(histgram[th:])

  # クラス1とクラス2の画素値の平均を計算
  if n1 == 0:
    mu1 = 0
  else:
    mu1 = sum([i * histgram[i] for i in range(0, th)]) / n1   
  if n2 == 0:
    mu2 = 0
  else:
    mu2 = sum([i * histgram[i] for i in range(th, 256)]) / n2

  # クラス1とクラス2の分散を計算
  if n1 == 0:
    sigma1 = 0
  else:
    sigma1 = sum([(i - mu1)**2 * histgram[i] for i in range(0, th)]) / n1
  if n2 == 0:
    sigma2 = 0
  else:
    sigma2 = sum([(i - mu2)**2 * histgram[i] for i in range(th, 256)]) / n2

  # クラス間分散
  s_b = (n1 * n2 * (mu1 - mu2) ** 2) / (n1 + n2)

  # クラス内分散
  s_w = (n1 * sigma1 + n2 * sigma2) / (n1 + n2)

  # クラス1とクラス2の間の分散（クラス間分散）の分子を計算
  s = n1 * n2 * (mu1 - mu2) ** 2
  # s = s_b / s_w

  # クラス間分散の分子が大きい時，クラス間分散の分子と閾値を記録
  if s > s_max[1]:
    s_max = [th, s]
    
# クラス間分散が最大のときの閾値を取得
threshold = s_max[0]
print("THRESHOLD =", threshold)

# 算出した閾値で二値化処理
img_bin_da = img.copy()
img_bin_da[img_bin_da < threshold] = 0
img_bin_da[img_bin_da >= threshold] = 255

# 表示
plt.imshow(img_bin_da, cmap="gray", clim=(0, 255))
plt.show()

## 二値画像処理

次に，二値化した画像を対象とする画像処理を行う．

まず，二値画像処理に使用する二値画像を用意する．
今回は，OpenCVの`threshold`関数を用いて判別分析法により二値画像を用意する．

In [ ]:
img = img_gray.copy()
th, img_bin = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
print("threshold:", th)

plt.imshow(img_bin, cmap="gray", clim=(0, 255))
plt.show()

### 収縮・膨張処理

収縮・膨張処理は二値画像のノイズ除去等に使用される処理である．

#### 収縮 (erosion)

背景または穴に接する対象画素のひとまわりを剥ぎ取る処理

In [ ]:
img = img_bin.copy()
kernel = np.ones((5, 5), np.uint8)
erosion = cv2.erode(img, kernel, iterations=1)

plt.imshow(erosion, cmap="gray", clim=(0, 255))

#### 膨張 (dilation)

背景または穴に接する対象画素のひとまわりを加える処理

In [ ]:
img = img_bin.copy()
kernel = np.ones((5, 5), np.uint8)
dilation = cv2.dilate(img, kernel, iterations=1)

plt.imshow(dilation, cmap="gray", clim=(0, 255))

#### オープニング (opening)

同じ回数だけ**収縮**して**膨張**する処理 

In [ ]:
img = img_bin.copy()
opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

plt.imshow(opening, cmap="gray", clim=(0, 255))

#### クロージング (closing)

同じ回数だけ**膨張**して**収縮**する処理

In [ ]:
img = img_bin.copy()
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

plt.imshow(closing, cmap="gray", clim=(0, 255))

### ラベリング

ラベリングは同じ連結成分を構成する画素に同じ番号を付与する処理である．

まず，各画素のラベルを保存するための配列`label`を用意する．
次に，for文を用いて1画素ずつラベルを割り当てていく．
注目画素が背景（白画素）の場合は，注目画素ではないため処理をスキップする．
注目画素の場合は，以下の手順でラベルをつける

1. 注目画素の上の画素がラベルを持つとき，そのラベルを注目画素に付与

  * 注目画素の左の画素がラベルを持ち，注目画素のラベルと異なるとき，左画素のラベルと同一の画素のラベルを注目画素のラベルに置き換える
  （ルックアップテーブルに保存し，あとでラベルを統合する方法もある）

3. 注目画素の上の画素が白画素で，左の画素がラベルを持つとき，そのラベルを注目画素に付与
4. 注目画素の上の左も白画素のとき，新しいラベルを注目画素に付与

In [ ]:
img = closing.copy()

label = np.zeros(img.shape, dtype=np.int32)

label_counter = 1
for h in range(label.shape[0]):
  for w in range(label.shape[1]):
    if img[h, w] == 255:
      continue
    
    # 2-1. 
    if label[h-1, w] != 0:
      label[h, w] = label[h-1, w]
      # 2-2.
      if label[h, w-1] != 0 and label[h, w-1] != label[h, w]:
        label[label == label[h, w-1]] = label[h, w]
      continue

    # 3.
    if label[h-1, w] == 0 and label[h, w-1] != 0:
      label[h, w] = label[h, w-1]
      continue

    # 4.
    if label[h-1, w] == 0 and label[h, w-1] == 0:
      label[h, w] = label_counter
      label_counter += 1
      continue

上のラベルの割り当てによって得られた結果を表示する．
Numpyの`unique`関数で配列内の要素の値の種類（ラベル）を取り出す．

その後，各ラベルにランダムに色を決定し，対応する画素に色付けをして表示する．
（色の区別がわかりにくい場合は，何度か実行すること）

この結果より，連結した領域をひとまとまりとしてラベルづけされていることがわかる．

In [ ]:
import random

labels = np.unique(label)

img_segment = np.zeros([label.shape[0], label.shape[1], 3], dtype=np.uint8)

for l in labels:
  if l != 0:
    color = (random.randint(50, 255), random.randint(50, 255), random.randint(50, 255))
    img_segment[label == l] = color

plt.figure(figsize=(16, 12))
plt.imshow(img_segment)
plt.show()

## 課題

* グレースケール変換時のRGBの比率を変更して，変換結果を確認すること
* pタイル法の比率を変更して，適切な比率を確認すること
* 縮小・膨張処理のkernelを変更して，変化を確認すること